# MultiplEYE preprocessing

In [1]:
from pathlib import Path
import polars as pl
import pandas as pd
import re
import json
import PIL
import math
import pickle
from sanity import extract_information_file_path, report_to_file
from report import check_gaze, check_metadata, check_events, report_to_file as report_meta
from functools import partial
from plot import plot_gaze, plot_main_sequence, load_data, preprocess
from stimulus import Stimulus, load_stimuli
from stimulus import load_stimuli

import pymovements as pm

In [2]:
# Create a dictionary with all the relevant information from the file path

In [ ]:
## NL testrun
path_asc_file = "data\\017_NL_NL_1_ET1_testrun_1733144369\\017nlnl1.asc"
stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\017_NL_NL_1_ET1_testrun_1733144369\logfiles\EXPERIMENT_LOGFILE_1_017_2024-12-02_1733144369.txt"

In [4]:
### Turkisch Data

path_asc_file = "data\\001_TR_TR_1_ET1\\001trtr1.asc"
stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_TR_TR_Istanbul_1_2025"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\001_TR_TR_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_001_2025-01-16_1737022316.txt"


In [11]:
local_file_part = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing"

vars_dict = extract_information_file_path(path_asc_file, local_file_part, stimulus_file_path, logfile_path)
vars_dict["asc_file"]


Match was found at 5-33: 001_TR_TR_1_ET1\001trtr1.asc
Group 1 found at 5-8: 001
Group 2 found at 9-11: TR
Group 3 found at 12-14: TR
Group 4 found at 15-16: 1
Group 5 found at 17-20: ET1
Group 6 found at -1--1: None
Group 7 found at 21-29: 001trtr1
C:\Users\saphi\PycharmProjects\multipleye-preprocessing\quality-report\output\TR_TR_Istanbul_1_2025
True


WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/001_TR_TR_1_ET1/001trtr1.asc')

In [13]:
vars_dict["stimuli_order"] 


[13, 7, 8, 12, 10, 9, 4, 3, 6, 11, 2, 1]

In [14]:
with open(vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_gaze.pkl", "rb") as f:
    gaze = pickle.load(f)
    
gaze.events.frame

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\saphi\\PycharmProjects\\multipleye-preprocessing\\quality-report\\output\\TR_TR_Istanbul_1_2025\\001trtr1_gazel.pkl'

In [27]:
### Create or load gaze dataframe from ASC file, with the provided lab configuration
### Only execute if no gaze dataframe is available
try:
    with open(vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_gaze.pkl", "rb") as f:
         gaze = pickle.load(f)
except FileNotFoundError:
        stimuli, lab_config = load_stimuli(vars_dict["stimulus_file_path"], vars_dict["lang"], vars_dict["country"], vars_dict["labnum"])
        gaze = load_data(vars_dict['asc_file'], lab_config)
        preprocess(gaze)
        ### save and load the gaze dataframe to pickle for later usage
        with open(vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_gaze.pkl", "wb") as f:
            pickle.dump(gaze, f)
        with open(vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_gaze.pkl", "rb") as f:
            gaze = pickle.load(f)
        
                
gaze.events.frame



trial,stimulus,screen,name,onset,offset,duration,location,amplitude,peak_velocity
str,str,str,str,i64,i64,i64,list[f64],f64,f64
"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",1185963,1186497,534,"[68.61514, 112.910467]",null,null
"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",1186530,1186717,187,"[111.012766, 110.398936]",null,null
"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",1186758,1186945,187,"[132.107979, 181.034043]",null,null
"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",1186970,1187117,147,"[115.966216, 151.737162]",null,null
"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",1187154,1187639,485,"[132.261934, 205.009877]",null,null
…,…,…,…,…,…,…,…,…,…
"""trial_10""","""PopSci_MultiplEYE_1""","""question_1131""","""saccade""",4378449,4378512,63,null,4.117215,104.896666
"""trial_10""","""PopSci_MultiplEYE_1""","""question_1131""","""saccade""",4378741,4378795,54,null,2.264978,60.012357
"""trial_10""","""PopSci_MultiplEYE_1""","""question_1131""","""saccade""",4378892,4378940,48,null,2.073484,56.06927


In [21]:
gaze.frame.filter(pl.col("activity") == "question").unique("screen")


time,pupil,activity,screen,practice,stimulus,trial,pixel,position,velocity
i64,f64,str,str,bool,str,str,list[f64],list[f64],list[f64]
1887514,755.0,"""question""","""question_8111""",false,"""Lit_Solaris_8""","""trial_1""","[1262.8, 929.3]","[15.248466, 11.12818]","[0.555458, -0.108777]"
1261650,851.0,"""question""","""question_13121""",true,"""Enc_WikiMoon_13""","""PRACTICE_trial_1""","[1274.1, 933.5]","[15.525235, 11.235292]","[-0.188187, -0.435624]"
3626882,703.0,"""question""","""question_4132""",false,"""Lit_Alchemist_4""","""trial_5""","[1249.2, 930.2]","[14.914396, 11.151139]","[-0.891229, -0.20315]"
2762722,778.0,"""question""","""question_10212""",false,"""Arg_PISACowsMilk_10""","""trial_3""","[1247.6, 926.2]","[14.875025, 11.04907]","[1.530864, 0.608406]"
1922273,818.0,"""question""","""question_8131""",false,"""Lit_Solaris_8""","""trial_1""","[1284.4, 926.4]","[15.776865, 11.054175]","[1.720514, -0.581945]"
…,…,…,…,…,…,…,…,…,…
4022150,795.0,"""question""","""question_6121""",false,"""Lit_MagicMountain_6""","""trial_7""","[632.2, 265.6]","[-0.995846, -6.292073]","[-0.537997, 0.199062]"
1278885,891.0,"""question""","""question_13131""",true,"""Enc_WikiMoon_13""","""PRACTICE_trial_1""","[1262.9, 937.7]","[15.250919, 11.342325]","[1.454413, 0.495623]"
2817041,847.0,"""question""","""question_10232""",false,"""Arg_PISACowsMilk_10""","""trial_3""","[1236.3, 945.1]","[14.596559, 11.530712]","[0.232841, 0.501403]"


In [22]:
gaze.frame.filter(pl.col("stimulus") == "PopSci_Caveman_12").filter(pl.col("activity") == "rating").unique(pl.col("screen"))

time,pupil,activity,screen,practice,stimulus,trial,pixel,position,velocity
i64,f64,str,str,bool,str,str,list[f64],list[f64],list[f64]
2251295,903.0,"""rating""","""familiarity_rating_screen_2""",false,"""PopSci_Caveman_12""","""trial_2""","[1280.8, 944.1]","[15.688987, 11.505269]","[1.132809, -0.115387]"
2247144,736.0,"""rating""","""familiarity_rating_screen_1""",false,"""PopSci_Caveman_12""","""trial_2""","[1276.8, 929.1]","[15.591256, 11.123077]","[0.775368, -0.005783]"
2254520,856.0,"""rating""","""subject_difficulty_screen""",false,"""PopSci_Caveman_12""","""trial_2""","[1281.0, 934.4]","[15.693871, 11.258235]","[-0.292637, 0.672089]"


In [23]:
### inspect Metadata
gaze._metadata

{'weekday': 'Thu',
 'month': 'Jan',
 'day': 16,
 'time': '14:26:20',
 'year': 2025,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v5.09 Nov 17 2015',
 'resolution': (1341, 1009),
 'tracking_mode': 'CR',
 'sampling_rate': 1000.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'L',
 'mount_configuration': {'mount_type': 'Binocular Tower Mount',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'binocular / monocular',
  'short_name': 'BTOWER'},
 'pupil_data_type': 'AREA',
 'version_number': '5.09',
 'model': 'EyeLink 1000 Plus',
 'datetime': datetime.datetime(2025, 1, 16, 14, 26, 20),
 'calibrations': [{'timestamp': '1091276.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '1440454.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '1973449.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '2381613.0',
   'num_points': '9',
   'typ

In [1]:
report_file_2 = open(f"{vars_dict['output_dir']}/{vars_dict['participant_abbr']}_report2.txt", "w", encoding="utf-8")
report = partial(report_meta, report_file=report_file_2)
check_gaze(gaze, report)    
check_metadata(gaze._metadata, report)

NameError: name 'vars_dict' is not defined

### preprocess gaze data to create event dataframe, which is used for plotting


In [28]:
gaze.events

shape: (14_022, 10)
┌────────────┬───────────┬───────────┬──────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ trial      ┆ stimulus  ┆ screen    ┆ name     ┆ … ┆ duration ┆ location  ┆ amplitude ┆ peak_velo │
│ ---        ┆ ---       ┆ ---       ┆ ---      ┆   ┆ ---      ┆ ---       ┆ ---       ┆ city      │
│ str        ┆ str       ┆ str       ┆ str      ┆   ┆ i64      ┆ list[f64] ┆ f64       ┆ ---       │
│            ┆           ┆           ┆          ┆   ┆          ┆           ┆           ┆ f64       │
╞════════════╪═══════════╪═══════════╪══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ PRACTICE_t ┆ Enc_WikiM ┆ page_1    ┆ fixation ┆ … ┆ 534      ┆ [68.61514 ┆ null      ┆ null      │
│ rial_1     ┆ oon_13    ┆           ┆          ┆   ┆          ┆ , 112.910 ┆           ┆           │
│            ┆           ┆           ┆          ┆   ┆          ┆ 467]      ┆           ┆           │
│ PRACTICE_t ┆ Enc_WikiM ┆ page_1    ┆ fixation ┆ … ┆ 187      ┆ [111.0

In [ ]:
check_events(gaze, report)

### Plotting


In [ ]:
plot_dir = vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_plots"
plot_dir.mkdir(exist_ok=True)
plot_main_sequence(gaze.events, plot_dir)
for stimulus in stimuli:
    print(f"Plotting {stimulus.name}...")
    plot_gaze(gaze, stimulus, plot_dir)


### Loading Logfile

In [ ]:
logfile= pl.read_csv(logfile_path, separator="\t")
logfile

In [ ]:
stimulus_frame = logfile.filter(
              (pl.col("stimulus_number") == f"{11}")
            )
stimulus_frame.write_csv("stimulus_frame.csv")
stimulus_frame

In [32]:
print(stimuli)

[Stimulus(id=1, name='PopSci_MultiplEYE', type='experiment', pages=[StimulusPage(number=1, text='Het MultiplEYE-project\r\n\r\nDe naam "MultiplEYE" is een woordspeling die "multilingualism" of "multiple languages" (Engels voor "meertaligheid") combineert met "eye" (Engels voor "oog") van "eye-tracking" (oogbewegingsmeting). MultiplEYE is een door de Europese Unie gefinancierde COST-actie. COST-acties zijn onderzoeksnetwerken die worden ondersteund door de European Cooperation in Science and Technology, kortweg COST. Als financieringsorganisatie ondersteunt COST ons groeiende netwerk van onderzoekers in heel Europa en daarbuiten, door financiële steun te verlenen voor het uitvoeren van verschillende netwerkactiviteiten.', image_path=WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024/stimuli_images_NL_NL_1/popsci_multipleye_id1_page_1_NL.png')), StimulusPage(number=2, text='Onder deze activiteiten zijn vergaderingen van werk

In [45]:
def check_all_screens_logfile(logfile: pl, stimuli: Stimulus):
    """ checking if all screens, where ET data is tracked are present in the log file"""
    for stimulus in stimuli:
        print(f"Checking {stimulus.name} in Logfile")
        trial_id = logfile.filter((pl.col("stimulus_number") == f"{stimulus.id}")).item(0, "trial_number") # get the trial number for the stimulus as ratingscreens don't have an entry in the stimulus_number column
      
        stimulus_frame = logfile.filter(
              (pl.col("trial_number") == f"{trial_id}")
            )
        #print(stimulus_frame)
        # check if all pages are present        
        for page in stimulus.pages:
            if f"{page.number}" not in stimulus_frame["page_number"].to_list():
                print(f"Missing page {stimulus.name} {page.number} in Logfile")  
                report_to_file(f" {stimulus.name}: Missing page{page.number} in Logfile")
        # check if all questions are present              
        for question in stimulus.questions:
            if f"{question.id}" not in stimulus_frame["page_number"].to_list() and f"{question.id[1:]}" not in stimulus_frame["page_number"].to_list():
                print(f"{stimulus.name}: Missing question_{question.id} in Logfile")
                report_to_file(f"{stimulus.name}: Missing question_{question.id} in Logfile")
               # print(stimulus_frame["screen"])
                
        for rating in stimulus.ratings:
            if f"{rating.name}" not in stimulus_frame["page_number"].to_list():
                print(f"{stimulus.name}: Missing rating screen {rating.name}")
                report_to_file(f"{stimulus.name}: Missing rating screen {rating.name} in Logfile")
               
check_all_screens_logfile(logfile, stimuli)

Checking PopSci_MultiplEYE in Logfile
Checking Ins_HumanRights in Logfile
Checking Ins_LearningMobility in Logfile
Checking Lit_Alchemist in Logfile
Checking Lit_MagicMountain in Logfile
Checking Lit_Solaris in Logfile
Checking Lit_BrokenApril in Logfile
Checking Arg_PISACowsMilk in Logfile
Checking Arg_PISARapaNui in Logfile
Checking PopSci_Caveman in Logfile
Checking Enc_WikiMoon in Logfile
Checking Lit_NorthWind in Logfile


## check if all pages screens ect are present in ASC file

In [46]:

def check_all_screens(gaze, stimuli):
    """ checking if all screens, where ET data is tracked are present in the gaze data frame, which is based on the ASC file
    it checks for all stimuli, if all pages and questions screens are present;it does not check for valditaion, calibration, instructions, etc.
    """
    for stimulus in stimuli:
        print(f"Checking {stimulus.name}")
        stimulus_frame = gaze.frame.filter(
              (pl.col("stimulus") == f"{stimulus.name}_{stimulus.id}")
            ).unique("screen")
        # check if all pages are present        
        for page in stimulus.pages:
            if f"page_{page.number}" not in stimulus_frame["screen"].to_list():
                print(f"Missing page {page.number}")  
                report_to_file(f"Missing page {page.number} in asc file", vars_dict["report_file"])
        # check if all questions are present              
        for question in stimulus.questions:
            if f"question_{question.id}" not in stimulus_frame["screen"].to_list() and f"question_{question.id[1:]}" not in stimulus_frame["screen"].to_list():
                print(f"Missing question_{question.id}")
                report_to_file(f"Missing question_{question.id} in asc file", vars_dict["report_file"])
                print(stimulus_frame["screen"])
                
        for rating in stimulus.ratings:
            if f"{rating.name}" not in stimulus_frame["screen"].to_list():
                print(f"Missing instruction {rating.name}")
                report_to_file(f"Missing rating {rating.name} in asc file", vars_dict["report_file"])
                                
check_all_screens(gaze, stimuli)

Checking PopSci_MultiplEYE
Checking Ins_HumanRights
Checking Ins_LearningMobility
Checking Lit_Alchemist
Checking Lit_MagicMountain
Checking Lit_Solaris
Checking Lit_BrokenApril
Checking Arg_PISACowsMilk
Checking Arg_PISARapaNui
Checking PopSci_Caveman
Checking Enc_WikiMoon
Checking Lit_NorthWind


## Checking the order of the experiment

In [32]:
def load_messages(asc_file: Path, REGEX):
    with open(asc_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    messages = []
    for line in lines:
        match = re.match(REGEX, line)
        if match:
            messages.append(match.groupdict())
    return messages

#REGEX = r'MSG\s+(?P<timestamp>\d+[.]?\d*)\s+(?P<message>\w+)'
REGEX = r'MSG\s+(?P<timestamp>\d+[.]?\d*)\s+(?P<message>.*)'

messages = load_messages(vars_dict["asc_file"], REGEX)

In [33]:
print(messages)

[{'timestamp': '820456.0', 'message': '!CMD select_parser_configuration 0'}, {'timestamp': '820461.0', 'message': 'var initial eye_used right'}, {'timestamp': '820547.0', 'message': 'welcome_screen'}, {'timestamp': '910748.0', 'message': 'informed_consent_screen'}, {'timestamp': '929556.0', 'message': 'start_experiment'}, {'timestamp': '929556.0', 'message': 'stimulus_order_version: 8'}, {'timestamp': '929556.0', 'message': 'showing_instruction_screen_1'}, {'timestamp': '929630.0', 'message': 'showing_instruction_screen_2'}, {'timestamp': '944501.0', 'message': 'showing_instruction_screen_3'}, {'timestamp': '955532.0', 'message': 'camera_setup_screen'}, {'timestamp': '1068593.0', 'message': 'DISPLAY_COORDS 0 0 1340 1008'}, {'timestamp': '1091276.0', 'message': '!CAL '}, {'timestamp': '1091276.0', 'message': '!CAL Calibration points:  '}, {'timestamp': '1091276.0', 'message': '!CAL -19.6, -44.5        -0,     23   '}, {'timestamp': '1091276.0', 'message': '!CAL -19.0, -60.8        -0,  

In [34]:
stimulus = stimuli[2]
stimulus.name, stimulus.pages[0]

('Ins_LearningMobility',
 StimulusPage(number=1, text='KOMİSYONDAN KONSEYE RAPOR: Öğrenme Hareketliliği Kriteri İlerleme Raporu\r\n\r\n1. GİRİŞ\r\n\r\nAvrupa Gençliğine Yatırım üzerine yapılan son Komisyon Bildirimi, öğrenme\r\nhareketliliğinin istihdam edilebilirlik ve aktif vatandaşlık üzerindeki olumlu\r\netkisini vurgulamaktadır. Başka bir ülkede öğrenim görmek, eğitim almak ve staj\r\nyapmak eşsiz bir deneyimdir ve yeni ufuklar açabilir.', image_path=WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/stimuli_MultiplEYE_TR_TR_Istanbul_1_2025/stimuli_images_TR_TR_1/ins_learningmobility_id3_page_1_TR.png')))

In [36]:

print(vars_dict["stimuli_order"])


[13, 7, 8, 12, 10, 9, 4, 3, 6, 11, 2, 1]


In [41]:
# check order in ASC file based on messages
def check_order(messages, stimuli):
    
    messages_only = [d.get('message') for d in messages]
    one_time_screens = ['welcome_screen', 'informed_consent_screen', 'start_experiment', 'stimulus_order_version', 'showing_instruction_screen_1', 'showing_instruction_screen_2', 'showing_instruction_screen_3', 'camera_setup_screen', 'practice_text_starting_screen', 'start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1','stop_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1', 'start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_2', 'stop_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_2','transition_screen', 'final_validation', 'show_final_screen', 'obligatory_break', 'obligatory_break_end']
    optional_screens = ['empty_screen', 'optional_break_screen', 'fixation_trigger:skipped_by_experimenter']
    reoccuring_screens = ['showing_subject_difficulty_screen', 'showing_familiarity_rating_screen_1','showing_familiarity_rating_screen_2']
    
    
    def _check_validation_screen(last_index, index_next_stimulus):
        if "validation_before_stimulus" not in messages_only[last_index:index_next_stimulus]:
            print(f"{stimulus.name}: Missing validation screen")
            report_to_file(f"{stimulus.name}: Missing validation screen in asc file", vars_dict["report_file"])
        else:
          #  print(f"{stimulus.name}: Validation screen found")
            val_index = messages_only[last_index:index_next_stimulus].index("validation_before_stimulus")
            
     
    def _get_stimulus(id, next_id=None):
        right_stimulus = None
        next_stimulus = None
        for stimulus in stimuli:
            if stimulus.id == id:
                right_stimulus = stimulus
            if stimulus.id == next_id:
                next_stimulus = stimulus
       
        return right_stimulus, next_stimulus
            
    def _check_instruction_screens(last_index, index_next_stimulus):
        for instruction in reoccuring_screens:
            if f"{instruction}" not in messages_only[last_index:index_next_stimulus]:
                print(f"Missing instruction {instruction}")
                report_to_file(f"Missing instruction {instruction} in asc file", vars_dict["report_file"])
           # else:
                #print(f"Rating {instruction} screens found")
    
                
    def _reoccuring_msg(trial):
        pattern = f"_trial_{trial}_stimulus_{stimulus.name}_{stimulus.id}"
        last_index = messages_only.index(f"start_recording{pattern}_page_1")
        last_timestamp = messages[last_index].get("timestamp")
        print(last_timestamp)
        if next_stimulus:
            index_next_stimulus = messages_only.index(f"start_recording_trial_{trial+1}_stimulus_{next_stimulus.name}_{next_stimulus.id}_page_1")
        else:
            index_next_stimulus = len(messages_only)-1
        next_timestamp = messages[index_next_stimulus].get("timestamp")
        print(next_timestamp)
        print(round(((float(next_timestamp)-float(last_timestamp))/60000), 2))
        report_to_file(f"Trial {trial}: {round(((float(next_timestamp)-float(last_timestamp))/60000), 2)} minutes", vars_dict["report_file"])
                
        msg_to_find = ["start_recording","screen_image_onset", "screen_image_offset", "stop_recording"]
    
            
        for page in stimulus.pages:
            for msg in msg_to_find:
                if msg.startswith("screen"):
                    current_pattern = f"page_{msg}"
                else:
                    current_pattern = f"{msg}{pattern}_page_{page.number}"
                    
                if current_pattern not in messages_only[last_index:index_next_stimulus]:
                    print(f"{stimulus.name}: Missing {current_pattern} Messages in ASC file")
                else:
                    last_index = messages_only[last_index:index_next_stimulus].index(current_pattern) + last_index
                            
        _check_instruction_screens(last_index, index_next_stimulus)  
       
        for question in stimulus.questions:
            for msg in msg_to_find:
                if msg.startswith("screen"): # get the cleaned questio_id (removed zero at the start, if present)
                    current_pattern = f"question_{msg}"
                else:
                    if question.id.startswith("0"):
                        question_id = question.id[1:]
                    else:
                        question_id = question.id
                        
                    current_pattern = f"{msg}{pattern}_question_{question_id}"
                    
                if current_pattern not in messages_only[last_index:index_next_stimulus]:
                    print(f"{stimulus.name}: Missing {current_pattern} Messages in ASC file")
                # else:
                  #  print(f"{stimulus.name}: {current_pattern} Messages is in ASC file")
                    
                    
        _check_validation_screen(last_index, index_next_stimulus)
                
    for trial, id in enumerate(vars_dict["stimuli_order"][2:]): # skip th e practice trials
        try:
            stimulus, next_stimulus = _get_stimulus(id, vars_dict["stimuli_order"][2:][trial+1]) # also get next stimulus to check for the reoccuring screens
        except IndexError:
            print("Last stimulus")
            stimulus, next_stimulus = _get_stimulus(id, None)
            
        print(f"Checking {stimulus.name}")
        _reoccuring_msg(trial+1) # add one because the first trial  has number 1 not 0 
    
                
    for one_time_screen in one_time_screens:
        if f"{one_time_screen}" not in messages_only:
            print(f"Missing screen {one_time_screen}")
            report_to_file(f"Missing one time screen {one_time_screen} in asc file", vars_dict["report_file"])
       # else:
        #    print(f"{one_time_screen} found")
            
    for optional_screen in optional_screens:
        if f"{optional_screen}" in messages_only:
            report_to_file(f"Optional screen {optional_screen} found in asc file", vars_dict["report_file"])
            print(f"{optional_screen} found")

check_order(messages, stimuli)


Checking Lit_Solaris
1470171.0
2059626.0
9.82
Checking PopSci_Caveman
2059626.0
2406977.0
5.79
Checking Arg_PISACowsMilk
2406977.0
3106545.0
11.66
Checking Lit_BrokenApril
3106545.0
3461194.0
5.91
Checking Lit_Alchemist
3461194.0
3828630.0
6.12
Checking Ins_LearningMobility
3828630.0
3970375.0
2.36
Checking Lit_MagicMountain
3970375.0
4071978.0
1.69
Checking Arg_PISARapaNui
4071978.0
4213222.0
2.35
Checking Ins_HumanRights
4213222.0
4299390.0
1.44
Last stimulus
Checking PopSci_MultiplEYE
4299390.0
4465758.0
2.77
PopSci_MultiplEYE: Missing validation screen
Missing screen stimulus_order_version
Missing screen start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1
Missing screen stop_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1
Missing screen start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_2
Missing screen stop_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_2
